In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import meerkat as mk
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

from domino.data.cxr import get_dp, build_cxr_df, get_cxr_activations, rle2mask
from domino.feedback import ScribbleModel

In [3]:
# Get a mosaic DataPanel with the data.
df = build_cxr_df.out(load=True)
dp = get_dp(df)
dp.head()

,image_id (NumpyArrayColumn),encoded_pixels (NumpyArrayColumn),pmx (NumpyArrayColumn),filepath (NumpyArrayColumn),chest_tube (NumpyArrayColumn),split (NumpyArrayColumn),gaze_seq (NumpyArrayColumn),gaze_heatmap (NumpyArrayColumn),gaze_max_visit (NumpyArrayColumn),gaze_unique (NumpyArrayColumn),gaze_time (NumpyArrayColumn),gaze_diffusivity (NumpyArrayColumn),expert_label (NumpyArrayColumn),index (ListColumn),input (CellColumn),img (CellColumn)
0,1.2.276.0.7230010.3.1.4.8323329.6904.151787520...,-1,0,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,NaN,train,nan,nan,NaN,NaN,NaN,NaN,NaN,'0',MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...,MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...
1,1.2.276.0.7230010.3.1.4.8323329.13666.15178752...,557374 2 1015 8 1009 14 1002 20 997 26 990 32 ...,1,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,NaN,train,nan,nan,NaN,NaN,NaN,NaN,NaN,'1',MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...,MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...
2,1.2.276.0.7230010.3.1.4.8323329.11028.15178752...,-1,0,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,NaN,train,nan,nan,NaN,NaN,NaN,NaN,NaN,'2',MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...,MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...
3,1.2.276.0.7230010.3.1.4.8323329.10366.15178752...,514175 10 1008 29 994 30 993 32 991 33 990 34 ...,1,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,NaN,train,nan,nan,NaN,NaN,NaN,NaN,NaN,'3',MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...,MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...
4,1.2.276.0.7230010.3.1.4.8323329.10016.15178752...,592184 33 976 58 956 73 941 88 926 102 917 109...,1,/media/4tb_hdd/siim/dicom-images-train/1.2.276...,NaN,train,nan,nan,NaN,NaN,NaN,NaN,NaN,'4',MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...,MedicalVolumeCell([PosixPath('/media/4tb_hdd/s...


In [4]:
train_dp = mk.DataPanel.read("/home/sabri/data/domino/feedback/06-24_khaled_cxr_fb.dp")
train_dp = mk.merge(train_dp, dp, how="inner", on="image_id")
train_dp["feedback_mask"] = train_dp["feedback_mask"][:]

In [5]:
train_dp = get_cxr_activations(
    dp=train_dp, 
    model_path="/home/common/datasets/cxr-tube/models/cxr_pmx_mimic_pretrained.pth"
)

In [18]:
scrib_model = ScribbleModel(threshold=0.3, strategy="mask_pos_v", activation_size=(14, 14))
scrib_model.fit(train_dp, activation_col="activation_block3")

In [9]:
test_dp = dp.lz[~np.isnan(dp["chest_tube"].data) & (dp["split"].data == "test")]
test_dp = get_cxr_activations(
    dp=test_dp, 
    model_path="/home/common/datasets/cxr-tube/models/cxr_pmx_mimic_pretrained.pth"
)

In [19]:
y_hat = scrib_model.predict(test_dp, activation_col="activation_block3")

In [20]:
roc_auc_score(test_dp["chest_tube"], y_hat)

0.8866083153220821

In [10]:
roc_auc_score(test_dp["chest_tube"], test_dp["probs"].data[:, -1])

0.8637429679476369

In [21]:
roc_auc_score(test_dp["chest_tube"], y_hat + test_dp["probs"].data[:, -1])

0.9092590605535713

In [23]:
test_dp["scribble_probs"] = y_hat

In [28]:
test_dp[["image_id", "scribble_probs", "chest_tube"]].lz[np.argsort(test_dp["scribble_probs"])][-20:]

,image_id (NumpyArrayColumn),scribble_probs (NumpyArrayColumn),chest_tube (NumpyArrayColumn),index (ListColumn)
0,1.2.276.0.7230010.3.1.4.8323329.12924.15178752...,0.769668,1.0,'11707'
1,1.2.276.0.7230010.3.1.4.8323329.10046.15178752...,0.775927,1.0,'9304'
2,1.2.276.0.7230010.3.1.4.8323329.32436.15178751...,0.782845,1.0,'3345'
3,1.2.276.0.7230010.3.1.4.8323329.12499.15178752...,0.785263,1.0,'540'
4,1.2.276.0.7230010.3.1.4.8323329.4135.151787518...,0.786105,1.0,'4037'
5,1.2.276.0.7230010.3.1.4.8323329.2820.151787517...,0.795010,1.0,'5513'
6,1.2.276.0.7230010.3.1.4.8323329.14356.15178752...,0.803570,1.0,'4292'
7,1.2.276.0.7230010.3.1.4.8323329.12188.15178752...,0.818865,1.0,'8425'
8,1.2.276.0.7230010.3.1.4.8323329.5257.151787518...,0.821486,1.0,'3082'
9,1.2.276.0.7230010.3.1.4.8323329.10076.15178752...,0.821993,1.0,'8850'


In [32]:
test_dp["scribble_acts"] = scrib_model.predict(test_dp, activation_col="activation_block3", reduction=None)

In [40]:
def vis_scribble_preds(dp):
    global_min = dp["scribble_acts"].min()
    global_max = dp["scribble_acts"].max()
    for row in dp[:30]:
       _, axes = plt.subplots(1, 2, figsize=(50,100))
       inp = np.array(row["img"])
       axes[0].imshow(inp, cmap="gray")
       axes[1].imshow(inp, cmap="gray")

       preds = np.array(row["scribble_acts"])
       preds = preds.repeat(
           inp.shape[0] / preds.shape[0], axis=0
       ).repeat(inp.shape[1] / preds.shape[1], axis=1)
       axes[1].imshow(
           preds,
           alpha=0.79, 
           cmap="PiYG",
           vmin=global_min,
           vmax=global_max
       )

In [ ]:
vis_scribble_preds(test_dp)